# Introduction

Notebook para fazer análise de modelos e de resultados de experimentos feitos e armazenados no Neptune

# Import Libraries

In [1]:
import os
import sys
import neptune.new as neptune
import zipfile
import shutil
import numpy as np
import pandas as pd

from datetime import datetime

from tqdm.notebook import tqdm

if '../..' not in sys.path:
    sys.path.insert(0, '../..')

import config

# Get Project Data

In [2]:
df = pd.read_csv('single_task_exps_data/icao-nets-training-2.csv')
print(df.shape)
df.head(23)

(23, 7)


,Id,Creation Time,Owner,Tags,properties/icao_reqs,properties/aligned,properties/gt_names
0,ICAO-186,2021-06-12 13:21:22,guilhermemg,"[""ground truths"",""vgg16"",""binary_output"",""adam...",['red_eyes'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
1,ICAO-185,2021-06-12 13:18:09,guilhermemg,"[""binary_output"",""ground truths"",""adamax"",""vgg...",['background'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
2,ICAO-184,2021-06-12 13:14:38,guilhermemg,"[""fvc"",""ground truths"",""binary_output"",""adamax...",['hair_eyes'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
3,ICAO-183,2021-06-12 13:11:25,guilhermemg,"[""ground truths"",""pixelation"",""binary_output"",...",['pixelation'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
4,ICAO-182,2021-06-12 13:08:26,guilhermemg,"[""binary_output"",""ground truths"",""vgg16"",""wash...",['washed_out'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
5,ICAO-181,2021-06-12 13:04:52,guilhermemg,"[""fvc"",""adamax"",""skin_tone"",""binary_output"",""g...",['skin_tone'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
6,ICAO-180,2021-06-12 13:03:24,guilhermemg,"[""adamax"",""fvc"",""ink_mark"",""binary_output"",""vg...",['ink_mark'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
7,ICAO-179,2021-06-12 13:00:08,guilhermemg,"[""binary_output"",""ground truths"",""vgg16"",""adam...",['blurred'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
8,ICAO-178,2021-06-12 12:56:45,guilhermemg,"[""ground truths"",""adamax"",""vgg16"",""sh_head"",""f...",['sh_head'],0.0,"{'train_validation': [], 'test': [], 'train_va..."
9,ICAO-177,2021-06-12 12:53:18,guilhermemg,"[""sh_face"",""fvc"",""ground truths"",""binary_outpu...",['sh_face'],0.0,"{'train_validation': [], 'test': [], 'train_va..."


# Download de Modelos

In [3]:
def process_gt_names(gt_names):
    gt_names_dict = {'train_validation_test': gt_names.split(':')[-1]}
    train_val_test = gt_names_dict['train_validation_test']
    train_val_test = train_val_test[:-3]
    train_val_test = train_val_test[3:]
    return train_val_test


def download_model(exp_id):
    try:
        neptune_run = None
        print(f'Experiment ID: {exp_id}')

        neptune_run = neptune.init(project='guilhermemg/icao-nets-training-2', 
                                   api_token=config.NEPTUNE_API_TOKEN,
                                   run=exp_id)

        ds = neptune_run['properties/gt_names'].fetch()
        alg = neptune_run['properties/aligned'].fetch()
        req = neptune_run['properties/icao_reqs'].fetch()

        ds = process_gt_names(ds)
        aligned = 'aligned' if bool(alg) == True else 'not_aligned'
        req = req[2:-2]
        
        print(f' Requisite: {req.upper()} | Dataset: {ds.upper()} | Aligned: {aligned}')
        
        dest_dir = f'prev_trained_models/single_task/{ds}_{aligned}/{req}/{exp_id}'
        print(f'Destination path: {dest_dir}')
        
        if os.path.exists(dest_dir):
            #shutil.rmtree(dest_dir)
            print('Model already downloaded!')
            print('-----------------------------')
            return
            
        os.makedirs(dest_dir)
        
        try:
            success_download = False
            neptune_run['artifacts/trained_model'].download(dest_dir)
            success_download = True

            with zipfile.ZipFile(os.path.join(dest_dir, 'trained_model.zip'), 'r') as zip_ref:
                zip_ref.extractall(dest_dir)

            os.remove(os.path.join(dest_dir, 'trained_model.zip'))
            shutil.move(os.path.join(dest_dir, 'trained_model', 'variables'), dest_dir)
            shutil.move(os.path.join(dest_dir, 'trained_model', 'saved_model.pb'), dest_dir)
            shutil.rmtree(os.path.join(dest_dir, 'trained_model'))

            print('.. Folders set')
        except Exception as ex:
            print(f'Experiment {exp_id} did not produce a trained model!')
        finally:
            if not success_download:
                shutil.rmtree(dest_dir)
            print('-----------------------------')
            return
            
    except Exception as e:
        raise e
    finally:
        if neptune_run is not None:
            neptune_run.stop()

            
for exp_id in df.Id.values:
    download_model(exp_id)            

Experiment ID: ICAO-186
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-186
 Requisite: RED_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/red_eyes/ICAO-186
.. Folders set
-----------------------------
Experiment ID: ICAO-185
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-185
 Requisite: BACKGROUND | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/background/ICAO-185
.. Folders set
-----------------------------
Experiment ID: ICAO-184
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-184
 Requisite: HAIR_EYES | Dataset: FVC | Aligned: not_aligned
Destination path: prev_trained_models/single_task/fvc_not_aligned/hair_eyes/ICAO-184
.. Folders set
-----------------------------
Experiment ID: ICAO-183
https://ui.neptune.ai/guilhermemg/icao-nets-training-2/e/ICAO-183
 Requisite: PIXELATION | Dataset: FVC | Aligned: not_aligned
Destinati